# Query Object


In [ ]:
#| default_exp query

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export
from cu_api import config
from cu_api.core import get_cf_options, prc_get_cf_fields
import pandas as pd
from fastcore.basics import patch
from typing import Union, List

In [ ]:
#| export

class Query:
    #"The `Query` object is used to store and process search parameters when using the Copper API. Since using the Copper API is not very straight forward, it acts as a human comprehensable semantic layer that cu_api then decodes into the format the the API expects."

    #  This class allows input of both native Copper field names or custom field IDs for search parameters.
    #  It processes these inputs to prepare them for querying the Copper API.

    def __init__(self):
        self._data = {}             # Raw data (Keys and Values) provided by the user
        self._processed_data = {}   # Keys and Values after they have been processed
        self._native_fields = []    # List of all of the native field keys
        self.outputs = []           # The columns that the user has specified they want in the outputed DataFrame

    def __repr__(self):
        return f"Search Object: \nInputs:  {self._data}, \nProcessed Data:  {self._processed_data} \nNative Fields used:{self._native_fields})"


## Setting Parameters

Users are able to pass parameters by settting setting which values they want for any field on company records

```python
Query['Field Name'] = ['Vales','I','want']

# Or, if you know the ID of the custome field, you can use that instead of the name

Query[8675309] = ['Jenny']

# You can do the same thing with field value if you know those as well:

Query['Custom Field'] = [667788,889900,112233]
```

In [ ]:
#| export

@patch
def __setitem__(self:Query, 
                key:str,  # Name of Copper field or the Custome Field ID
                value: list[str] # Field values that you want to be included
                ):
    """ Allows user to set accepte values for a Copper field. 
    """
    self._data[key] = value
    self._process_input(key, value)

## Processing Inputs

After parameters are inputed by the user, work is done behind the scene to check if this inputs are valid and pre-prossess before being used in a search.

### 1. Ensuring the keys provide by the user are valid field names or valid field ids

In [ ]:
#| export

def _check_key(key):
    """
    Function to ensure that 'keys' are custom field id as an interager
    """
    prc_get_cf_fields()
    CF_ID_LOOKUP = getattr(config,'CF_ID_LOOKUP')
    CUSTOM_FIELDS_DICT = getattr(config,'CUSTOM_FIELDS_DICT')
    
    if isinstance(key,str) and key in CF_ID_LOOKUP.keys():
        return CF_ID_LOOKUP.get(key)
    elif isinstance(key,int) and key in CUSTOM_FIELDS_DICT.keys():
        return key
    elif isinstance(key,str):
        try:
            int_key = int(key)
            if int_key in CUSTOM_FIELDS_DICT.keys():
                return int_key
        except ValueError:
            pass
    
    raise ValueError(f"Provided key '{key}' is not a valid custom field name(str) or custom field id(int)")

In [ ]:
show_doc(_check_key,title_level=4)

---

[source](https://github.com/cooper-richason/cu_api/blob/main/cu_api/query.py#L42){target="_blank" style="float:right; font-size:smaller"}

#### _check_key

>      _check_key (key)

*Function to ensure that 'keys' are custom field id as an interager*

### 2. Ensuring values provided by user are valid names or valid value ids

In [ ]:
#| export

def _check_value(key:int, value):
    """
    Function to convert provided custom field values to their corresponding IDs.

    This function ensures that the provided 'values' are the IDs for the given custom field key 'key'.
    """
    if isinstance(value, str): value = [value]
    if isinstance(value, int): value = [value]

    def find_key_by_value(dictionary, search_value):

        for key, value in dictionary.items():
            if value == search_value:
                return key

    cf_options = get_cf_options(key)
    Updated_List = []

    for item in value:
        if isinstance(item,str) and item in cf_options.values():
            id_value = find_key_by_value(cf_options, item)
            Updated_List.append(id_value)
        elif isinstance(item,int) and item in cf_options.keys():
            Updated_List.append(item)
        elif isinstance(item,str):
            try:
                int_key = int(item)
                if int_key in cf_options.keys():
                    Updated_List.append(int_key)
            except ValueError:
                continue
    
    return list(set(Updated_List))

In [ ]:
show_doc(_check_value,title_level=4)

---

[source](https://github.com/cooper-richason/cu_api/blob/main/cu_api/query.py#L65){target="_blank" style="float:right; font-size:smaller"}

#### _check_value

>      _check_value (key:int, value)

*Function to convert provided custom field values to their corresponding IDs.

This function ensures that the provided 'values' are the IDs for the given custom field key 'key'.*

### Combining these into Function

The `_process_input` function performs both `_check_key` and `_check_value` operations before storing the processed key and value in the _processed_data dictionary.

In [ ]:
#| export

@patch
def _process_input(self:Query, key, value):
    """Default processing function that stores the length of the value."""

    if isinstance(key,list) or key not in ['id','name','address','assignee_ids','contact_type_id',
                'phone_number','city','state','postal_code','email_domains','tags','due_date','reminder_date','completed_date']:
        self.outputs.append(key)
        key = _check_key(key) 
        value = _check_value(key,value)
    elif key not in self._native_fields:
        self._native_fields.append(key)

    self._processed_data[key] = value

In [ ]:
show_doc(Query._process_input,title_level=4)

---

[source](https://github.com/cooper-richason/cu_api/blob/main/cu_api/query.py#L108){target="_blank" style="float:right; font-size:smaller"}

#### Query._process_input

>      Query._process_input (key, value)

*Default processing function that stores the length of the value.*

## Methods and Functions

In [ ]:
#| export

@patch
def __getitem__(self:Query, key):
    if key in self._data:               return self._data[key]
    elif key in self._processed_data:   return self._processed_data[key]
    else:                               raise KeyError(f"Key '{key}' not found in provided Query.")

 
@patch
def get_input(self:Query, key, default=None):
    """Get the original value provided by user."""
    return self._data.get(key, default)

@patch
def get_output(self:Query, key, default=None):
    """Get the processed value."""
    return self._processed_data.get(key, default)

In [ ]:
#| export

@patch
def inputs(self:Query):
    return self._data.items()

@patch
def input_keys(self:Query):
    return self._data.keys()

@patch
def input_values(self:Query):
    return self._data.values()

@patch
def items(self:Query):
    return self._processed_data.items()

@patch
def keys(self:Query):
    return list(self._processed_data.keys())

@patch
def values(self:Query):
    return self._processed_data.values()

In [ ]:
#| export

def _process_query(Query):
    """
    Function to take in a Query object and outputs items needed to search Copper.

    Input
    -----
    
    Query (object) containing the desired search parameters and information on desired outputs

    Outputs
    ------
    
    Native_Params - list[dict] for search parameters related to fields native to copper records types.

        Example: [{'state':'CA'}]

    CF_Params - list[dict] or search parameters related to custom fields on copper records. These have a special format dictated by Copper.

        Example: {"custom_field_definition_id": 588393, "value":[1806073]}

    Output_CFs - list of desired custom fields for the output dataframe
    """
    Native_Params = {}
    CF_Params = []
    Outputs = Query.keys()
    list_non_native = Query.keys() #starts out as all items and Native fields are removed in next step. Maybe should change this

    if Query._native_fields: 
        for item in Query._native_fields:
            Native_Params[item] = Query.get_output(item)
            list_non_native.remove(item)
    
    if list_non_native:
        for item in list_non_native:
            CF_Params.append({"custom_field_definition_id": item,"value":Query.get_output(item)})
    
    for item in Query.outputs:
        Outputs.append(_check_key(item))
        
    return Native_Params, CF_Params, Outputs

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()